# Finetune ruGPT3Small on essays

## Install enviroment

In [ ]:
!pip3 install urllib3==1.25.4

In [ ]:
!pip3 install transformers==2.8.0

     |████████████████████████████████| 573kB 11.9MB/s 
     |████████████████████████████████| 1.1MB 48.4MB/s 
     |████████████████████████████████| 890kB 51.3MB/s 
     |████████████████████████████████| 102kB 13.5MB/s 
     |████████████████████████████████| 3.7MB 46.8MB/s 
     |████████████████████████████████| 6.8MB 36.9MB/s 
     |████████████████████████████████| 71kB 11.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=ab09b00c4a287688ca19a9218d07a4cb140c0e3be240a131c759adf95daee119
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
  Created wheel for boto3: filename=boto3-1.16.17-py2.py3-none-any.whl size=128453 sha256=21b217169bde055826fd5173f472ee51ceb25767657ad261b4493492587e12ec
  Stored in directory: /root/.cache/pip/wheels/10/15/9a/ca25640e6aac0c28af4f111b09aea8f915e7c98caca3a6d326
Successfully built sacremoses boto3


In [ ]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/pretrain_transformers.py

--2020-11-13 06:26:17--  https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/pretrain_transformers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34150 (33K) [text/plain]
Saving to: ‘pretrain_transformers.py’

pretrain_transforme 100%[===================>]  33.35K  --.-KB/s    in 0s      

2020-11-13 06:26:17 (133 MB/s) - ‘pretrain_transformers.py’ saved [34150/34150]



In [ ]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py

--2020-11-13 06:26:17--  https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10474 (10K) [text/plain]
Saving to: ‘generate_transformers.py’

generate_transforme 100%[===================>]  10.23K  --.-KB/s    in 0s      

2020-11-13 06:26:17 (78.3 MB/s) - ‘generate_transformers.py’ saved [10474/10474]



In [ ]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Writing setup.sh


In [ ]:
!sh setup.sh

Cloning into 'apex'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 7456 (delta 0), reused 0 (delta 0), pack-reused 7455
Receiving objects: 100% (7456/7456), 13.91 MiB | 24.39 MiB/s, done.
Resolving deltas: 100% (5036/5036), done.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-wlpqk04n
Created temporary directory: /tmp/pip-req-tracker-sz2q85ya
Created requirements tracker '/tmp/pip-req-tracker-sz2q85ya'
Created temporary directory: /tmp/pip-install-ongsc7aw
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-1cxrv_b2
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-sz2q85ya'
    Running setup.py (path:/tmp/pip-req-build-1cxrv_b2/setup.py) egg_info for pac

## Add data to colab
Add essays file from google dirve:
* Add [file](https://drive.google.com/file/d/10ZsjTeaoihYA80n1G40O5YZmaGw0yOXk/view?usp=sharing) to your own drive
* And mount drive to colab

In [ ]:
!wget https://raw.githubusercontent.com/balajibalasubramanian/Kaggle-Toxic-Comments-Challenge/master/data/train.csv

--2020-11-13 06:32:18--  https://raw.githubusercontent.com/balajibalasubramanian/Kaggle-Toxic-Comments-Challenge/master/data/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68802655 (66M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]  65.62M   288MB/s    in 0.2s    

2020-11-13 06:32:19 (288 MB/s) - ‘train.csv’ saved [68802655/68802655]



In [ ]:
data_path = "train.csv"

## Prepare data

In [ ]:
import pandas as pd
data = pd.read_csv(data_path)
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
import numpy as np
import random

In [ ]:
random.seed(1234)
np.random.seed(1234)

In [ ]:
data.shape

(159571, 8)

In [ ]:
val_ind = random.sample(range(data.shape[0]), 15000)

In [ ]:
train = [data.iloc[i]['comment_text'] for i in range(len(data)) if i not in val_ind]
valid = [data.iloc[i]['comment_text'] for i in range(len(data)) if i in val_ind]

In [ ]:
len(train), len(valid)

(144571, 15000)

In [ ]:
with open("train.txt", "w") as file:
    file.write("\n".join(train))

In [ ]:
with open("valid.txt", "w") as file:
    file.write("\n".join(valid))

## Run finetuning
The following code download our model and tokenizer from transformers and finetune model essays.

This took aroung ten minutes and obtain perplexity = tensor(13.8065)

In [ ]:
!python pretrain_transformers.py \
    --output_dir=toxic_model \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=train.txt \
    --do_eval \
    --fp16 \
    --eval_data_file=valid.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 1 \
    --block_size 2048 \
    --overwrite_output_dir

Streaming output truncated to the last 5000 lines.
Iteration:  45% 3831/8574 [42:00<50:12,  1.57it/s]
Iteration:  45% 3832/8574 [42:01<50:17,  1.57it/s]
Iteration:  45% 3833/8574 [42:01<50:14,  1.57it/s]
Iteration:  45% 3834/8574 [42:02<50:17,  1.57it/s]
Iteration:  45% 3835/8574 [42:03<50:16,  1.57it/s]
Iteration:  45% 3836/8574 [42:03<50:10,  1.57it/s]
Iteration:  45% 3837/8574 [42:04<50:15,  1.57it/s]
Iteration:  45% 3838/8574 [42:04<50:11,  1.57it/s]
Iteration:  45% 3839/8574 [42:05<50:13,  1.57it/s]
Iteration:  45% 3840/8574 [42:06<50:13,  1.57it/s]
Iteration:  45% 3841/8574 [42:06<50:14,  1.57it/s]
Iteration:  45% 3842/8574 [42:07<50:14,  1.57it/s]
Iteration:  45% 3843/8574 [42:08<50:08,  1.57it/s]
Iteration:  45% 3844/8574 [42:08<50:10,  1.57it/s]
Iteration:  45% 3845/8574 [42:09<50:08,  1.57it/s]
Iteration:  45% 3846/8574 [42:10<50:11,  1.57it/s]
Iteration:  45% 3847/8574 [42:10<50:08,  1.57it/s]
Iteration:  45% 3848/8574 [42:11<50:02,  1.57it/s]
Iteration:  45% 3849/8574 [42:1

## Check our model

In [ ]:
!python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=essays_model \
    --k=5 \
    --p=0.95 \
    --length=500

python3: can't open file 'generate_transformers.py': [Errno 2] No such file or directory
